# Dbpedia and Caligraph---Reading files and preprocessing

In [16]:
import os, json
import numpy as np
import time, gc, sys
gc.enable()
base_path = !pwd
base_path = base_path[0]
list_files = [base_path+"/data/caligraph/"+f for f in os.listdir(base_path+"/data/caligraph/") if os.path.isfile(base_path+"/data/caligraph/"+f)]

In [17]:
sys.path.append('Baselines/GCN-Align/src/openea/')
from modules.finding.alignment import greedy_alignment

2022-03-18 17:46:14.146208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 17:46:14.146273: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
list_files

['/home/nkouagou/Documents/Universal_Embeddings/data/caligraph/caligraph-instance-to-dbpedia-mappings.nt']

In [4]:
with open(list_files[0]) as file:
    caligraph2dbpedia_mappings = file.readlines()

In [5]:
def get_map(mapping):
    x,_,y,_ = mapping.split()
    return x.strip('<>'), y.strip('<>')

In [6]:
caligraph2dbpedia_mappings = dict(map(lambda x: get_map(x), caligraph2dbpedia_mappings))

In [7]:
list(caligraph2dbpedia_mappings.items())[13]

('http://caligraph.org/resource/Cameroon_sheep',
 'http://dbpedia.org/resource/Cameroon_sheep')

In [8]:
#dbpedia2caligraph_mappings = {value: key for key,value in caligraph2dbpedia_mappings.items()}

In [9]:
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

In [10]:
word_vectors_caligraph = KeyedVectors.load("./Caligraph_Dbpedia/caligraph/caligraph-v211_500_4_sg_200_vectors.kv", mmap='r')

In [11]:
word_vectors_dbpedia = KeyedVectors.load("./Caligraph_Dbpedia/dbpedia/dbpedia.kv", mmap='r')

### There are mismatches between entity IRIs in 'caligraph2dbpedia_mappings' and those in the computed embeddings, see below. We will write a function that fixes it.

In [12]:
def repair_namespace(iri, kg='dbpedia'):
    if kg == 'dbpedia':
        if 'owl#' in iri:
            return iri
        iri = iri.replace('dbr:', 'http://dbpedia.org/resource/')
        return 'http://dbpedia.org/resource/' + iri.split('/')[-1]
    elif kg == 'caligraph':
        if 'owl#' in iri or 'ontology' in iri:
            return iri
        return 'http://caligraph.org/resource/' + iri.split('/')[-1]         

In [13]:
#Emb_keys_db = set(map(lambda t: repair_namespace(t), word_vectors_dbpedia.key_to_index.keys()))

In [14]:
#Emb_keys_cal = set(map(lambda t: repair_namespace(t, 'caligraph'), word_vectors_caligraph.key_to_index.keys()))

In [15]:
from tqdm import tqdm

### Creating entity to vector maps

In [16]:
entity2vec_db = {}
entity2vec_cal = {}

In [22]:
#word_vectors_dbpedia.index2entity

In [23]:
for ent in tqdm(word_vectors_dbpedia.index2entity):
    try:
        entity2vec_db[repair_namespace(ent)] = np.array(word_vectors_dbpedia.get_vector(ent)).astype(np.float16)
    except KeyError:
        if repair_namespace(ent) in entity2vec_db:
            entity2vec_db.pop(repair_namespace(ent))

100%|██████████| 15048578/15048578 [03:16<00:00, 76532.99it/s]


In [24]:
for ent in tqdm(word_vectors_caligraph.index2entity):
    try:
        entity2vec_cal[repair_namespace(ent, 'caligraph')] = np.array(word_vectors_caligraph.get_vector(ent)).astype(np.float16)
    except KeyError:
        if repair_namespace(ent) in entity2vec_cal:
            entity2vec_cal.pop(repair_namespace(ent))

100%|██████████| 16429696/16429696 [03:45<00:00, 72877.73it/s]


In [25]:
del word_vectors_dbpedia, word_vectors_caligraph
gc.collect()

31478801

In [26]:
#new_aligned_entity_dict = dict()
#
#for key, value in tqdm(caligraph2dbpedia_mappings.items()):
#    if key in entity2vec_cal and value in entity2vec_db:
#        new_aligned_entity_dict.update({key: value})
#
#
#print('There are ', len(new_aligned_entity_dict), ' aligned entities with available embeddings')

with open('Caligraph_Dbpedia/caligraph2dbpediaalignment.json', 'r') as file:
    new_aligned_entity_dict = json.load(file)

In [ ]:
#new_aligned_entity_dict

In [5]:
#with open('Caligraph_Dbpedia/list_merged_entities_cal_db.txt') as file:
#    entities = file.read().split('\t')

# Computing aligned KG embeddings with a simple neural network

In [ ]:
import torch, torch.nn as nn
from torch.optim import Adam

In [ ]:
class AlignmentModel(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.emb_dim = emb_dim
        self.R = torch.nn.Parameter(
                         torch.tensor(np.random.uniform(-1, 1, (emb_dim, emb_dim)),
                         dtype=torch.float, requires_grad=True))
        
    def forward(self, s):
        return torch.mm(s, self.R)

In [24]:
def get_batch(S, T, batch_size=128):
    for i in range(0, S.shape[0]-batch_size+1, batch_size):
        yield S[i:i+batch_size], T[i:i+batch_size]


def train(model, lr, epochs, S, T, batch_size=128):
    optimizer = Adam(model.parameters(), lr=lr)
    Loss = nn.MSELoss()
    for epoch in range(epochs):
        loss_epoch = 0
        for S_batch, T_batch in tqdm(get_batch(S, T, batch_size=batch_size), total=S.shape[0]//batch_size):
            proj = model(S_batch)
            loss = Loss(proj, T_batch)
            loss_epoch += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Loss epoch {epoch} : {loss_epoch}')

In [30]:
#with open('Caligraph_Dbpedia/caligraph2dbpediaalignment.json', 'w') as file:
#    json.dump(new_aligned_entity_dict, file, indent=3)

# Computing aligned KG embeddings using Orthogonal Procrustes

In [27]:
from sklearn.model_selection import train_test_split
from scipy.spatial import procrustes
from scipy.linalg import orthogonal_procrustes

## Get the embedding matrices of aligned an non-aligned entities

In [28]:
def get_source_and_target_matrices(alignment_dict, entity2vec1, entity2vec2, given_test_set=None, emb_dim=200, test_size=0.1):
    """This function takes the dictionary of aligned entities between two KGs and their corresponding embeddings (as entity to vector dictionaries)
    and returns S, T, S_eval, T_eval, and R defined as follows:
    
    -- S: Normalized and scaled large subset of the source embeddings, i.e. the matrix of aligned entity embeddings in the first knowledge graph
    
    -- T: Normalized and scaled large subset of the matrix of aligned entity embeddings in the second knowledge graph
    
    -- S_eval and T_eval are portions of S and T sampled for evaluation if test_size > 0
    
    -- R: The rotation matrix that most closely maps S to T, i.e. ||A@S-T|| is minimized
    
    The mean and standard deviation of S, T are also returned
    """
    if test_size > 0:
        if given_test_set is None:
            train_ents, eval_ents = train_test_split(list(alignment_dict.keys()), test_size=test_size, random_state=42)
        else:
            eval_ents = given_test_set
            train_ents = list(set(alignment_dict.keys())-set(eval_ents))
    else:
        train_ents = alignment_dict.keys()
        
    S = np.empty((len(train_ents), emb_dim))
    T = np.empty((len(train_ents), emb_dim))
    if test_size > 0:
        S_eval = np.empty((len(eval_ents), emb_dim))
        T_eval = np.empty((len(eval_ents), emb_dim))

    for i, key in tqdm(enumerate(train_ents), total=len(train_ents), desc='Computing S and T'):
        S[i] = entity2vec1[key] if isinstance(entity2vec1, dict) else entity2vec1.loc[key].values
        T[i] = entity2vec2[alignment_dict[key]] if isinstance(entity2vec2, dict) else entity2vec2.loc[alignment_dict[key]].values
        
    if test_size > 0:
        for i, key in tqdm(enumerate(eval_ents), total=len(eval_ents), desc='Computing S_eval and T_eval'):
            S_eval[i] = entity2vec1[key] if isinstance(entity2vec1, dict) else entity2vec1.loc[key].values
            T_eval[i] = entity2vec2[alignment_dict[key]] if isinstance(entity2vec2, dict) else entity2vec2.loc[alignment_dict[key]].values
        
    print('\nNow computing R...')
    # Center and scale data
    mean_S = S.mean(axis=0)
    mean_T = T.mean(axis=0)
    scale_S = np.sqrt(((S-mean_S)**2).sum()/S.shape[0]) # scale, see https://en.wikipedia.org/wiki/Procrustes_analysis
    scale_T = np.sqrt(((T-mean_T)**2).sum()/T.shape[0])
    print('Scale S: ', scale_S)
    
    t0 = time.time()
    R, loss = orthogonal_procrustes((S-mean_S)/scale_S, (T-mean_T)/scale_T, check_finite=True)
    print('\nCompleted after '+str(time.time()-t0)+' seconds')
    print('Alignment loss: ', loss)
    if test_size > 0:
        return scale_S, scale_T, mean_S, mean_T, (S-mean_S)/scale_S, (T-mean_T)/scale_T, (S_eval-mean_S)/scale_S, (T_eval-mean_T)/scale_T, R
        #return scale_S, scale_T, mean_S, mean_T, S, T, S_eval, T_eval, R
    else:
        return scale_S, scale_T, mean_S, mean_T, (S-mean_S)/scale_S, (T-mean_T)/scale_T, R


In [29]:
def get_non_aligned_entity_embedding_matrices(alignment_dict, entity2vec1, entity2vec2, scale_S, scale_T, mean_S, mean_T, emb_dim=200):
    """
    Inputs the dictionary of aligned entities between two KGs and their corresponding embeddings, and returns the normalized embedding matrices of 
    
    non-aligned entities
    """
    A_neg_S = np.empty((len(entity2vec1)-len(alignment_dict), emb_dim))
    keys = sorted(set(entity2vec1.keys() if isinstance(entity2vec1, dict) else entity2vec1.index)-set(alignment_dict.keys()))
    for i, key in tqdm(enumerate(keys), total=A_neg_S.shape[0], desc='Computing A_neg_S...'):
        A_neg_S[i] = entity2vec1[key] if isinstance(entity2vec1, dict) else entity2vec1.loc[key].values
    
    B_neg_T = np.empty((len(entity2vec2)-len(alignment_dict), emb_dim))
    keys = sorted(set(entity2vec2.keys() if isinstance(entity2vec2, dict) else entity2vec2.index)-set(alignment_dict.values()))
    for i, key in tqdm(enumerate(keys), total=B_neg_T.shape[0], desc='Computing B_neg_T...'):
        B_neg_T[i] = entity2vec2[key] if isinstance(entity2vec2, dict) else entity2vec2.loc[key].values
        
    return (A_neg_S-mean_S)/scale_S, (B_neg_T-mean_T)/scale_T
    

In [30]:
from sklearn.neighbors import NearestNeighbors
import random

In [31]:
def evaluate_alignment_knn(S_eval, T_eval, R, assume_known=False, hit_values = [1, 3, 10]):
    """The function takes the evaluation sets, i.e. correct alignments that were left out, and returns the hits@ and MRR results w.r.t. correct alignments
    
    --assume_known. A boolean variable. When set to True, the alignment results are computed using the fact that the test links are known
    
    """
    print('#'*50)
    print('Evaluation started...')
    print('#'*50)
    model = NearestNeighbors(n_neighbors=S_eval.shape[0], n_jobs=-1)
    print('Fitting 1...')
    model.fit(T_eval)
    print('Predicting 1...')
    if assume_known:
        preds = model.kneighbors((S_eval@R+T_eval)/2, n_neighbors=S_eval.shape[0], return_distance=False)
    else:
        preds = model.kneighbors(S_eval, n_neighbors=S_eval.shape[0], return_distance=False)
    Hits1 = np.zeros(len(hit_values))
    MRR1 = 0.0
    for i in tqdm(range(S_eval.shape[0]), total=S_eval.shape[0]):
        pred_idx = (preds[i]==i).nonzero()[0][0] # if i in preds[i] else S_eval.shape[0]
        MRR1 += (1./(pred_idx+1))
        for j in range(len(Hits1)):
            if pred_idx < hit_values[j]:
                Hits1[j] += 1.0/S_eval.shape[0]
    MRR1 = MRR1/S_eval.shape[0]
    
    model = NearestNeighbors(n_neighbors=S_eval.shape[0], n_jobs=-1)
    print('\nFitting 2...')
    if assume_known:
        model.fit((S_eval@R+T_eval)/2)
    else:
        model.fit(S_eval)
    print('Predicting 2...')
    preds = model.kneighbors(T_eval, n_neighbors=S_eval.shape[0], return_distance=False)
    Hits2 = np.zeros(len(hit_values))
    MRR2 = 0.0
    for i in tqdm(range(S_eval.shape[0]), total=S_eval.shape[0]):
        pred_idx = (preds[i]==i).nonzero()[0][0] # if i in preds[i] else S_eval.shape[0]
        MRR2 += (1./(pred_idx+1))
        for j in range(len(Hits2)):
            if pred_idx < hit_values[j]:
                Hits2[j] += 1.0/S_eval.shape[0]
    MRR2 = MRR2/S_eval.shape[0]
    
    Hits = (Hits1+Hits2)/2
    MRR = (MRR1+MRR2)/2
    print()
    print(', '.join([f'Hits@{hit_values[it]}: {Hits[it]}' for it in range(len(Hits))]+[f'MRR: {MRR}']))

## Evaluate, compute and store universal embeddings

In [32]:
_, _, _, _, _, _, S_eval, T_eval, R = get_source_and_target_matrices(new_aligned_entity_dict,\
                                                                                           entity2vec_cal, entity2vec_db, test_size=0.1)

Computing S_eval and T_eval: 100%|██████████| 374524/374524 [00:03<00:00, 112547.68it/s]



Now computing R...
Scale S:  3.398880398754246

Completed after 24.65157413482666 seconds
Alignment loss:  683398.1352021103


### Evaluation on validation data

In [34]:
hit_values = [1, 5, 10]

In [1]:
_, _, _, _ = greedy_alignment(embed1=S_eval@R, embed2=(S_eval@R+T_eval)/2, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

In [ ]:
#evaluate_alignment_knn(S_eval.astype(np.float16), T_eval.astype(np.float16), R.astype(np.float16), hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting...
Predicting...


In [22]:
list_merged_entities = sorted(set(entity2vec_cal.keys())-set(new_aligned_entity_dict.keys())) +\
sorted(set(entity2vec_db.keys())-set(new_aligned_entity_dict.values())) + \
list(new_aligned_entity_dict.keys())
with open('Caligraph_Dbpedia/list_merged_entities_cal_db.txt', 'w') as file:
    file.write('\t'.join(list_merged_entities))
#del list_merged_entities

In [24]:
scale_S, scale_T, mean_S, mean_T, S, T, R = get_source_and_target_matrices(new_aligned_entity_dict,\
                                                                           entity2vec_cal, entity2vec_db, test_size=0.0)

Computing S and T: 100%|██████████| 3745232/3745232 [00:38<00:00, 97640.99it/s] 



Now computing R...
Scale S:  3.399197955650588

Completed after 21.183332681655884 seconds
Alignment loss:  760397.9543931824


### Simple neural network

In [55]:
#model = AlignmentModel(200)
#lr = 0.01
#epochs = 100
#batch_size = 512

In [9]:
#train(model, lr, epochs, torch.Tensor(S), torch.Tensor(T), batch_size)

### Evaluation on training data

In [ ]:
#evaluate_alignment_knn(S, T, R, hit_values=[1, 3, 5, 10])

In [25]:
A_neg_S, B_neg_T = get_non_aligned_entity_embedding_matrices(new_aligned_entity_dict, entity2vec_cal, \
                                                             entity2vec_db, scale_S, scale_T, mean_S, mean_T)

Computing B_neg_T...: 100%|██████████| 6280733/6280733 [00:24<00:00, 256525.47it/s]


In [26]:
del entity2vec_cal, entity2vec_db
gc.collect()

0

In [27]:
# compute every s_i as (s_i@R+t_i)/2
S = (S@R + T)/2
del T
gc.collect()
Universal_Emb = np.concatenate([A_neg_S@R, B_neg_T, S], axis=0)

In [32]:
Universal_Emb.shape

(22695966, 200)

In [ ]:
np.save('Caligraph_Dbpedia/Universal_Emb.npy', Universal_Emb)

In [ ]:
Universal_Emb.shape

In [ ]:
del A_neg_S, B_neg_T, S, T, R, Universal_Emb
gc.collect()

## Using Shallom embeddings for Fr-En Dbpedia

In [1]:
import torch, pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial import procrustes
from scipy.linalg import orthogonal_procrustes
import time, gc
from sklearn.neighbors import NearestNeighbors
import random
from tqdm import tqdm
gc.enable()
random.seed(42)
import sys, os
sys.path.append('Baselines/GCN-Align/src/openea/')
from modules.finding.alignment import greedy_alignment

2022-03-18 18:14:35.343826: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 18:14:35.343882: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def get_source_and_target_matrices(alignment_dict, entity2vec1, entity2vec2, given_test_set=None, emb_dim=200, test_size=0.1):
    """This function takes the dictionary of aligned entities between two KGs and their corresponding embeddings (as entity to vector dictionaries)
    and returns S, T, S_eval, T_eval, and R defined as follows:
    
    -- S: Normalized and scaled large subset of the source embeddings, i.e. the matrix of aligned entity embeddings in the first knowledge graph
    
    -- T: Normalized and scaled large subset of the matrix of aligned entity embeddings in the second knowledge graph
    
    -- S_eval and T_eval are portions of S and T sampled for evaluation if test_size > 0
    
    -- R: The rotation matrix that most closely maps S to T, i.e. ||A@S-T|| is minimized
    
    The mean and standard deviation of S, T are also returned
    """
    if test_size > 0:
        if given_test_set is None:
            train_ents, eval_ents = train_test_split(list(alignment_dict.keys()), test_size=test_size, random_state=42)
        else:
            eval_ents = given_test_set
            train_ents = list(set(alignment_dict.keys())-set(eval_ents))
    else:
        train_ents = alignment_dict.keys()
    
    S = np.empty((len(train_ents), emb_dim))
    T = np.empty((len(train_ents), emb_dim))
    if test_size > 0:
        S_eval = np.empty((len(eval_ents), emb_dim))
        T_eval = np.empty((len(eval_ents), emb_dim))

    for i, key in tqdm(enumerate(train_ents), total=len(train_ents), desc='Computing S and T'):
        S[i] = entity2vec1[key] if isinstance(entity2vec1, dict) else entity2vec1.loc[key].values
        T[i] = entity2vec2[alignment_dict[key]] if isinstance(entity2vec2, dict) else entity2vec2.loc[alignment_dict[key]].values
        
    if test_size > 0:
        for i, key in tqdm(enumerate(eval_ents), total=len(eval_ents), desc='Computing S_eval and T_eval'):
            S_eval[i] = entity2vec1[key] if isinstance(entity2vec1, dict) else entity2vec1.loc[key].values
            T_eval[i] = entity2vec2[alignment_dict[key]] if isinstance(entity2vec2, dict) else entity2vec2.loc[alignment_dict[key]].values
        
    print('\nNow computing R...')
    # Center and scale data
    mean_S = S.mean(axis=0)
    mean_T = T.mean(axis=0)
    scale_S = np.sqrt(((S-mean_S)**2).sum()/S.shape[0]) # scale, see https://en.wikipedia.org/wiki/Procrustes_analysis
    scale_T = np.sqrt(((T-mean_T)**2).sum()/T.shape[0])
    print('Scale S: ', scale_S)
    
    t0 = time.time()
    R, loss = orthogonal_procrustes((S-mean_S)/scale_S, (T-mean_T)/scale_T, check_finite=True)
    print('\nCompleted after '+str(time.time()-t0)+' seconds')
    print('Alignment loss: ', loss)
    
    if test_size > 0:
        return scale_S, scale_T, mean_S, mean_T, (S-mean_S)/scale_S, (T-mean_T)/scale_T, (S_eval-mean_S)/scale_S, (T_eval-mean_T)/scale_T, R
        #return scale_S, scale_T, mean_S, mean_T, S, T, S_eval, T_eval, R
    else:
        return scale_S, scale_T, mean_S, mean_T, (S-mean_S)/scale_S, (T-mean_T)/scale_T, R
    

In [3]:
def get_non_aligned_entity_embedding_matrices(alignment_dict, entity2vec1, entity2vec2, scale_S, scale_T, mean_S, mean_T, emb_dim=200):
    """
    Inputs the dictionary of aligned entities between two KGs and their corresponding embeddings, and returns the normalized embedding matrices of 
    
    non-aligned entities
    """
    A_neg_S = np.empty((len(entity2vec1)-len(alignment_dict), emb_dim))
    keys = sorted(set(entity2vec1.keys() if isinstance(entity2vec1, dict) else entity2vec1.index)-set(alignment_dict.keys()))
    for i, key in tqdm(enumerate(keys), total=A_neg_S.shape[0], desc='Computing A_neg_S...'):
        A_neg_S[i] = entity2vec1[key] if isinstance(entity2vec1, dict) else entity2vec1.loc[key].values
    
    B_neg_T = np.empty((len(entity2vec2)-len(alignment_dict), emb_dim))
    keys = sorted(set(entity2vec2.keys() if isinstance(entity2vec2, dict) else entity2vec2.index)-set(alignment_dict.values()))
    for i, key in tqdm(enumerate(keys), total=B_neg_T.shape[0], desc='Computing B_neg_T...'):
        B_neg_T[i] = entity2vec2[key] if isinstance(entity2vec2, dict) else entity2vec2.loc[key].values
        
    return (A_neg_S-mean_S)/scale_S, (B_neg_T-mean_T)/scale_T
    

In [4]:
def evaluate_alignment_knn(S_eval, T_eval, R, assume_known=False, hit_values = [1, 3, 10]):
    """The function takes the evaluation sets, i.e. correct alignments that were left out, and returns the hits@ and MRR results w.r.t. correct alignments
    
    --assume_known. A boolean variable. When set to True, the alignment results are computed using the fact that the test links are known
    
    """
    print('#'*50)
    print('Evaluation started...')
    print('#'*50)
    model = NearestNeighbors(n_neighbors=S_eval.shape[0], n_jobs=-1)
    print('Fitting 1...')
    model.fit(T_eval)
    print('Predicting 1...')
    if assume_known:
        preds = model.kneighbors((S_eval@R+T_eval)/2, n_neighbors=S_eval.shape[0], return_distance=False)
    else:
        preds = model.kneighbors(S_eval, n_neighbors=S_eval.shape[0], return_distance=False)
    Hits1 = np.zeros(len(hit_values))
    MRR1 = 0.0
    for i in tqdm(range(S_eval.shape[0]), total=S_eval.shape[0]):
        pred_idx = (preds[i]==i).nonzero()[0][0] # if i in preds[i] else S_eval.shape[0]
        MRR1 += (1./(pred_idx+1))
        for j in range(len(Hits1)):
            if pred_idx < hit_values[j]:
                Hits1[j] += 1.0/S_eval.shape[0]
    MRR1 = MRR1/S_eval.shape[0]
    
    model = NearestNeighbors(n_neighbors=S_eval.shape[0], n_jobs=-1)
    print('\nFitting 2...')
    if assume_known:
        model.fit((S_eval@R+T_eval)/2)
    else:
        model.fit(S_eval)
    print('Predicting 2...')
    preds = model.kneighbors(T_eval, n_neighbors=S_eval.shape[0], return_distance=False)
    Hits2 = np.zeros(len(hit_values))
    MRR2 = 0.0
    for i in tqdm(range(S_eval.shape[0]), total=S_eval.shape[0]):
        pred_idx = (preds[i]==i).nonzero()[0][0] # if i in preds[i] else S_eval.shape[0]
        MRR2 += (1./(pred_idx+1))
        for j in range(len(Hits2)):
            if pred_idx < hit_values[j]:
                Hits2[j] += 1.0/S_eval.shape[0]
    MRR2 = MRR2/S_eval.shape[0]
    
    Hits = (Hits1+Hits2)/2
    MRR = (MRR1+MRR2)/2
    print()
    print(', '.join([f'Hits@{hit_values[it]}: {Hits[it]}' for it in range(len(Hits))]+[f'MRR: {MRR}']))

## Dbpedia 15K V1

In [5]:
EnFr_shallom_embs_v1 = pd.read_csv('Shallom_EnFr_15K_V1/Shallom_entity_embeddings.csv')

In [6]:
Fr_shallom_embs_v1 = EnFr_shallom_embs_v1[EnFr_shallom_embs_v1['Unnamed: 0'].apply(lambda x: 'fr.dbpedia.org' in x)]

In [7]:
En_shallom_embs_v1 = EnFr_shallom_embs_v1.iloc[np.setdiff1d(np.arange(EnFr_shallom_embs_v1.shape[0]),\
                                                            np.array(Fr_shallom_embs_v1.index))].set_index('Unnamed: 0')

In [8]:
Fr_shallom_embs_v1 = Fr_shallom_embs_v1.set_index('Unnamed: 0')

In [9]:
En_shallom_embs_v1.head(3)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/resource/Jean_Raoux_(soldier),0.212930,-0.456497,-0.804834,-0.044535,0.363582,-0.178872,0.385574,-0.230661,0.052410,-0.681095,...,0.315135,0.187490,0.047470,0.199877,-0.045452,-0.219743,0.397735,-0.400206,-0.561423,-0.075315
http://dbpedia.org/resource/Casually_Dressed_&_Deep_in_Conversation,-0.040372,0.139974,0.110328,-0.021151,0.499682,-0.227622,-0.223775,0.232769,0.114076,0.240751,...,0.085454,-0.053321,0.221335,-0.010698,0.285245,-0.098031,-0.010921,-0.012190,-0.076274,0.084391
http://dbpedia.org/resource/Resolve_(song),0.093407,0.785593,0.767466,0.389821,0.748004,-0.499349,0.325547,0.308278,0.554449,0.535334,...,0.245889,0.659989,0.651668,-0.110942,0.624555,-0.312246,0.011697,-0.722449,-0.116795,0.297614


In [10]:
with open('OpenEA_dataset_v1.1/EN_FR_15K_V1/ent_links') as file:
    en_to_fr_ents_v1 = file.read().strip().split('\n')
en_to_fr_ents_v1 = dict([line.split('\t') for line in en_to_fr_ents_v1])

In [11]:
with open('OpenEA_dataset_v1.1/EN_FR_15K_V1/721_5fold/1/test_links') as file:
    test_set = file.read().strip().split('\n')
test_set = [line.split('\t')[0] for line in test_set]

In [12]:
len(test_set)

10500

In [13]:
_, _, _, _, _, _, S_eval, T_eval, R = get_source_and_target_matrices(en_to_fr_ents_v1,\
                                                En_shallom_embs_v1, Fr_shallom_embs_v1, given_test_set=test_set, emb_dim=300, test_size=0.1)

Computing S_eval and T_eval: 100%|██████████| 10500/10500 [00:01<00:00, 5304.06it/s]



Now computing R...
Scale S:  7.206529678496196

Completed after 0.11171865463256836 seconds
Alignment loss:  1746.1492506830377


In [14]:
 hit_values=[1, 5, 10]

In [17]:
alignment_rest, _, _, _ = greedy_alignment(embed1=(S_eval@R+T_eval)/2, embed2=T_eval, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [79.171 88.381 91.076]%, mr = 29.987, mrr = 0.833329, time = 5.845 s 


In [16]:
_, _, _, _ = greedy_alignment(embed1=T_eval, embed2=(S_eval@R+T_eval)/2, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [81.8   85.886 87.162]%, mr = 164.615, mrr = 0.837038, time = 5.797 s 


In [32]:
evaluate_alignment_knn(S_eval, T_eval, R, assume_known=True, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


100%|██████████| 10500/10500 [00:00<00:00, 43776.57it/s]



Fitting 2...
Predicting 2...


100%|██████████| 10500/10500 [00:00<00:00, 42865.71it/s]


Hits@1: 0.8048571428572998, Hits@3: 0.8532380952382791, Hits@5: 0.8713333333335272, Hits@10: 0.8911904761906811, MRR: 0.8351833422747456


In [30]:
scale_S, scale_T, mean_S, mean_T, S, T, R = get_source_and_target_matrices(en_to_fr_ents_v1,\
                                                                                    En_shallom_embs_v1, Fr_shallom_embs_v1, emb_dim=300, test_size=0.0)

Computing S and T: 100%|██████████| 15000/15000 [00:03<00:00, 4429.51it/s]



Now computing R...
Scale S:  7.246380617560009

Completed after 0.17420744895935059 seconds
Alignment loss:  5129.786850424131


### On training data

In [31]:
evaluate_alignment_knn(S, T, R, assume_known=True, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


100%|██████████| 15000/15000 [00:00<00:00, 35695.17it/s]



Fitting 2...
Predicting 2...


100%|██████████| 15000/15000 [00:00<00:00, 34506.40it/s]


Hits@1: 0.8275999999999284, Hits@3: 0.8683333333332572, Hits@5: 0.882966666666589, Hits@10: 0.900566666666587, MRR: 0.8533851901211056


### Get non aligned entity embeddings

In [18]:
A_neg_S, B_neg_T = get_non_aligned_entity_embedding_matrices(en_to_fr_ents_v1, En_shallom_embs_v1, \
                                                             Fr_shallom_embs_v1, scale_S, scale_T, mean_S, mean_T, emb_dim=300)

Computing A_neg_S...: 0it [00:00, ?it/s]
Computing B_neg_T...: 0it [00:00, ?it/s]


In [19]:
# compute every s_i as (s_i@R+t_i)/2
S = (S@R + T)/2
del T
gc.collect()
Universal_Emb = np.concatenate([A_neg_S@R, B_neg_T, S], axis=0)

In [20]:
Universal_Emb.shape

(15000, 300)

In [21]:
np.save('Shallom_EnFr_15K_V1/Universal_Emb.npy', Universal_Emb)

In [12]:
list_merged_entities = sorted(set(En_shallom_embs_v1.index)-set(en_to_fr_ents_v1.keys())) +\
sorted(set(Fr_shallom_embs_v1.index)-set(en_to_fr_ents_v1.values())) + \
list(en_to_fr_ents_v1.keys())
print('Total: ', len(list_merged_entities))
with open('Shallom_EnFr_15K_V1/list_merged_entities_db_fr_en.txt', 'w') as file:
    file.write('\t'.join(list_merged_entities))
del list_merged_entities

Total:  15000


In [23]:
with open('Shallom_EnFr_15K_V1/english2french.txt', 'w') as file:
    json.dump(en_to_fr_ents_v1, file, ensure_ascii=False)

In [24]:
with open('Shallom_EnFr_15K_V1/french2english.txt', 'w') as file:
    json.dump({value:key for key,value in en_to_fr_ents_v1.items()}, file, ensure_ascii=False)

## Dbpedia 15K V2

In [19]:
EnFr_shallom_embs_v2 = pd.read_csv('Shallom_EnFr_15K_V2/Shallom_entity_embeddings.csv')

In [20]:
Fr_shallom_embs_v2 = EnFr_shallom_embs_v2[EnFr_shallom_embs_v2['Unnamed: 0'].apply(lambda x: 'fr.dbpedia.org' in x)]

In [21]:
En_shallom_embs_v2 = EnFr_shallom_embs_v2.iloc[np.setdiff1d(np.arange(EnFr_shallom_embs_v2.shape[0]),\
                                                            np.array(Fr_shallom_embs_v2.index))].set_index('Unnamed: 0')

In [22]:
Fr_shallom_embs_v2 = Fr_shallom_embs_v2.set_index('Unnamed: 0')

In [23]:
En_shallom_embs_v2.head(3)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/resource/High_Rhulain,-0.188881,0.315095,0.308269,-0.626206,-0.246237,-0.234675,0.194418,-0.466016,0.507792,-0.421602,...,-0.358771,-0.163102,0.117546,-0.575519,0.333001,-0.031201,0.354539,0.154133,-0.543737,-0.172065
http://dbpedia.org/resource/The_Ultimate_Death_Worship,-0.064648,0.240301,-0.364626,-0.270227,0.007151,0.033877,-0.013944,0.019905,0.075945,-0.135914,...,-0.278595,0.075917,0.188317,-0.253808,0.111861,0.188110,0.112244,0.218195,-0.120202,0.280531
http://dbpedia.org/resource/1932_Monaco_Grand_Prix,-0.832162,1.881694,0.928277,-0.659394,-0.277395,-2.025897,-0.496428,-0.328969,0.608205,-1.468092,...,-1.501762,-0.841657,-0.315155,-1.317987,1.216074,0.325797,-0.414880,-0.784873,-1.022442,-0.222932


In [24]:
with open('OpenEA_dataset_v1.1/EN_FR_15K_V2/ent_links') as file:
    en_to_fr_ents_v2 = file.read().strip().split('\n')
en_to_fr_ents_v2 = dict([line.split('\t') for line in en_to_fr_ents_v2])

In [25]:
with open('OpenEA_dataset_v1.1/EN_FR_15K_V2/721_5fold/1/test_links') as file:
    test_set = file.read().strip().split('\n')
test_set = [line.split('\t')[0] for line in test_set]

In [26]:
_, _, _, _, _, _, S_eval, T_eval, R = get_source_and_target_matrices(en_to_fr_ents_v2,\
                                                En_shallom_embs_v2, Fr_shallom_embs_v2, given_test_set=test_set, emb_dim=300, test_size=0.1)

Computing S_eval and T_eval: 100%|██████████| 10500/10500 [00:01<00:00, 5252.75it/s]



Now computing R...
Scale S:  10.211858760175003

Completed after 0.0732266902923584 seconds
Alignment loss:  1670.6351378751656


In [27]:
hit_values = [1, 5, 10]

In [28]:
_, _, _, _ = greedy_alignment(embed1=(S_eval@R+T_eval)/2, embed2=T_eval, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [81.343 89.867 92.486]%, mr = 12.991, mrr = 0.853062, time = 5.926 s 


In [29]:
_, _, _, _ = greedy_alignment(embed1=T_eval, embed2=(S_eval@R+T_eval)/2, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [88.238 91.181 92.114]%, mr = 75.184, mrr = 0.896596, time = 5.765 s 


In [40]:
evaluate_alignment_knn(S_eval, T_eval, R, assume_known=True, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


100%|██████████| 10500/10500 [00:00<00:00, 45043.12it/s]



Fitting 2...
Predicting 2...


100%|██████████| 10500/10500 [00:00<00:00, 44546.73it/s]


Hits@1: 0.8479047619049428, Hits@3: 0.8912380952383002, Hits@5: 0.905238095238308, Hits@10: 0.9230000000002226, MRR: 0.8748289524502817


In [13]:
scale_S, scale_T, mean_S, mean_T, S, T, R = get_source_and_target_matrices(en_to_fr_ents_v2,\
                                                                                    En_shallom_embs_v2, Fr_shallom_embs_v2, emb_dim=300, test_size=0.0)

Computing S and T: 100%|██████████| 15000/15000 [00:02<00:00, 5526.73it/s]



Now computing R...
Scale S:  10.233623721278317

Completed after 0.10644793510437012 seconds
Alignment loss:  4933.507408174066


In [42]:
evaluate_alignment_knn(S, T, R, assume_known=True, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


100%|██████████| 15000/15000 [00:00<00:00, 34976.59it/s]



Fitting 2...
Predicting 2...


100%|██████████| 15000/15000 [00:00<00:00, 35894.81it/s]


Hits@1: 0.867499999999924, Hits@3: 0.9034333333332534, Hits@5: 0.9146999999999188, Hits@10: 0.9303666666665837, MRR: 0.8897409102164076


In [14]:
A_neg_S, B_neg_T = get_non_aligned_entity_embedding_matrices(en_to_fr_ents_v2, En_shallom_embs_v2, \
                                                             Fr_shallom_embs_v2, scale_S, scale_T, mean_S, mean_T, emb_dim=300)

Computing A_neg_S...: 0it [00:00, ?it/s]
Computing B_neg_T...: 0it [00:00, ?it/s]


In [47]:
# compute every s_i as (s_i@R+t_i)/2
S = (S@R + T)/2
del T
gc.collect()
Universal_Emb = np.concatenate([A_neg_S@R, B_neg_T, S], axis=0)

In [48]:
Universal_Emb.shape

(15000, 300)

In [49]:
np.save('Shallom_EnFr_15K_V2/Universal_Emb.npy', Universal_Emb)

In [50]:
list_merged_entities = sorted(set(En_shallom_embs_v2.index)-set(en_to_fr_ents_v2.keys())) +\
sorted(set(Fr_shallom_embs_v2.index)-set(en_to_fr_ents_v2.values())) + \
list(en_to_fr_ents_v2.keys())
print('Total: ', len(list_merged_entities))
with open('Shallom_EnFr_15K_V2/list_merged_entities_db_fr_en.txt', 'w') as file:
    file.write('\t'.join(list_merged_entities))
del list_merged_entities

Total:  15000


In [51]:
with open('Shallom_EnFr_15K_V2/english2french.txt', 'w') as file:
    json.dump(en_to_fr_ents_v2, file, ensure_ascii=False)

In [52]:
with open('Shallom_EnFr_15K_V2/french2english.txt', 'w') as file:
    json.dump({value:key for key,value in en_to_fr_ents_v2.items()}, file, ensure_ascii=False)

## Fr-En Dbpedia 100K V1

In [30]:
EnFr_shallom_embs_v1 = pd.read_csv('Shallom_EnFr_100K_V1/Shallom_entity_embeddings.csv')

In [31]:
Fr_shallom_embs_v1 = EnFr_shallom_embs_v1[EnFr_shallom_embs_v1['Unnamed: 0'].apply(lambda x: 'fr.dbpedia.org' in x)]

In [32]:
En_shallom_embs_v1 = EnFr_shallom_embs_v1.iloc[np.setdiff1d(np.arange(EnFr_shallom_embs_v1.shape[0]),\
                                                            np.array(Fr_shallom_embs_v1.index))].set_index('Unnamed: 0')

In [33]:
Fr_shallom_embs_v1 = Fr_shallom_embs_v1.set_index('Unnamed: 0')

In [34]:
En_shallom_embs_v1.head(3)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/resource/Early_Edition,0.297347,-0.105210,0.133634,0.123629,0.246443,0.050322,0.330174,0.045672,-0.254961,0.101142,...,0.037443,-0.316077,-0.131114,-0.028953,-0.571384,0.473524,-0.302664,-0.136760,0.118570,0.621989
http://dbpedia.org/resource/Above_the_Law_(group),0.369008,-0.166957,0.071992,-0.633617,-0.897680,0.198830,0.373352,0.634033,-0.760533,-1.018539,...,-0.041860,-0.272072,-0.463082,-0.034017,0.413505,0.353298,-0.936489,0.449733,0.527858,-0.059133
http://dbpedia.org/resource/Belmont-Broye,-0.133468,0.032227,-0.218811,-0.056487,-0.049560,-0.050581,-0.064817,0.064291,-0.062564,-0.036485,...,0.138861,-0.049767,-0.013609,0.055437,0.038148,-0.015300,0.027329,-0.134478,-0.043584,-0.078838


In [35]:
with open('OpenEA_dataset_v1.1/EN_FR_100K_V1/ent_links') as file:
    en_to_fr_ents_v1 = file.read().strip().split('\n')
en_to_fr_ents_v1 = dict([line.split('\t') for line in en_to_fr_ents_v1])

In [36]:
with open('OpenEA_dataset_v1.1/EN_FR_100K_V1/721_5fold/1/test_links') as file:
    test_set = file.read().strip().split('\n')
test_set = [line.split('\t')[0] for line in test_set]

In [37]:
_, _, _, _, _, _, S_eval, T_eval, R = get_source_and_target_matrices(en_to_fr_ents_v1,\
                                                En_shallom_embs_v1, Fr_shallom_embs_v1, given_test_set=test_set, emb_dim=300, test_size=0.1)

Computing S_eval and T_eval: 100%|██████████| 70000/70000 [00:13<00:00, 5131.37it/s]



Now computing R...
Scale S:  4.01483484955182

Completed after 0.213700532913208 seconds
Alignment loss:  8501.940040621292


In [38]:
hit_values = [1, 5, 10]

In [39]:
_, _, _, _ = greedy_alignment(embed1=(S_eval@R+T_eval)/2, embed2=T_eval, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [65.74  76.874 80.584]%, mr = 159.250, mrr = 0.709728, time = 294.290 s 


In [40]:
_, _, _, _ = greedy_alignment(embed1=T_eval, embed2=(S_eval@R+T_eval)/2, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [76.884 81.727 83.286]%, mr = 1319.369, mrr = 0.791513, time = 298.184 s 


In [53]:
evaluate_alignment_knn(S_eval, T_eval, R, assume_known=True, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


100%|██████████| 70000/70000 [00:06<00:00, 10325.36it/s]



Fitting 2...
Predicting 2...


100%|██████████| 70000/70000 [00:33<00:00, 2061.23it/s] 


Hits@1: 0.7131214285714701, Hits@3: 0.7713285714287358, Hits@5: 0.793007142857353, Hits@10: 0.8193500000002657, MRR: 0.7506204284835016


In [13]:
scale_S, scale_T, mean_S, mean_T, S, T, R = get_source_and_target_matrices(en_to_fr_ents_v1,\
                                                                                    En_shallom_embs_v1, Fr_shallom_embs_v1, emb_dim=300, test_size=0.0)

Computing S and T: 100%|██████████| 100000/100000 [00:19<00:00, 5131.95it/s]



Now computing R...
Scale S:  4.016705119503967

Completed after 0.6553835868835449 seconds
Alignment loss:  26959.679985501716


In [ ]:
evaluate_alignment_knn(S, T, R, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


In [14]:
A_neg_S, B_neg_T = get_non_aligned_entity_embedding_matrices(en_to_fr_ents_v1, En_shallom_embs_v1, \
                                                             Fr_shallom_embs_v1, scale_S, scale_T, mean_S, mean_T, emb_dim=300)

Computing A_neg_S...: 0it [00:00, ?it/s]
Computing B_neg_T...: 0it [00:00, ?it/s]


In [15]:
# compute every s_i as (s_i@R+t_i)/2
S = (S@R + T)/2
del T
gc.collect()
Universal_Emb = np.concatenate([A_neg_S@R, B_neg_T, S], axis=0)

In [16]:
Universal_Emb.shape

(100000, 300)

In [17]:
np.save('Shallom_EnFr_100K_V1/Universal_Emb.npy', Universal_Emb)

In [18]:
list_merged_entities = sorted(set(En_shallom_embs_v1.index)-set(en_to_fr_ents_v1.keys())) +\
sorted(set(Fr_shallom_embs_v1.index)-set(en_to_fr_ents_v1.values())) + \
list(en_to_fr_ents_v1.keys())
print('Total: ', len(list_merged_entities))
with open('Shallom_EnFr_100K_V1/list_merged_entities_db_fr_en.txt', 'w') as file:
    file.write('\t'.join(list_merged_entities))
del list_merged_entities

Total:  100000


In [19]:
with open('Shallom_EnFr_100K_V1/english2french.txt', 'w') as file:
    json.dump(en_to_fr_ents_v1, file, ensure_ascii=False)

In [20]:
with open('Shallom_EnFr_100K_V1/french2english.txt', 'w') as file:
    json.dump({value:key for key,value in en_to_fr_ents_v1.items()}, file, ensure_ascii=False)

## Fr-En Dbpedia 100K V2

In [41]:
EnFr_shallom_embs_v2 = pd.read_csv('Shallom_EnFr_100K_V2/Shallom_entity_embeddings.csv')

In [42]:
Fr_shallom_embs_v2 = EnFr_shallom_embs_v2[EnFr_shallom_embs_v2['Unnamed: 0'].apply(lambda x: 'fr.dbpedia.org' in x)]

In [43]:
En_shallom_embs_v2 = EnFr_shallom_embs_v2.iloc[np.setdiff1d(np.arange(EnFr_shallom_embs_v2.shape[0]),\
                                                            np.array(Fr_shallom_embs_v2.index))].set_index('Unnamed: 0')

In [44]:
Fr_shallom_embs_v2 = Fr_shallom_embs_v2.set_index('Unnamed: 0')

In [45]:
En_shallom_embs_v2.head(3)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/resource/Sailing_to_Philadelphia,-0.107208,-0.463760,0.234986,-0.300843,-0.297771,0.157350,0.063645,0.219653,-0.373847,-0.517257,...,0.067203,-0.013326,0.009563,0.097662,-0.050666,0.292337,-0.091069,0.141930,0.412293,0.111441
http://dbpedia.org/resource/Horde_(band),-0.031448,0.542308,0.141856,-0.131928,-0.129672,0.056684,0.103640,0.017796,-0.080704,-0.920316,...,0.044921,0.069479,0.162442,0.144002,-0.108662,-0.412152,0.096725,0.008034,-0.273330,-0.185328
http://dbpedia.org/resource/Nessbeal,0.199559,-0.110438,0.225365,-0.145648,0.400099,-0.134547,0.287914,0.045707,0.731766,0.462888,...,0.217795,-0.313416,-0.295566,-0.239344,0.248340,0.115179,-0.185138,0.092450,-0.689202,0.140009


In [46]:
with open('OpenEA_dataset_v1.1/EN_FR_100K_V2/ent_links') as file:
    en_to_fr_ents_v2 = file.read().strip().split('\n')
en_to_fr_ents_v2 = dict([line.split('\t') for line in en_to_fr_ents_v2])

In [47]:
with open('OpenEA_dataset_v1.1/EN_FR_100K_V2/721_5fold/1/test_links') as file:
    test_set = file.read().strip().split('\n')
test_set = [line.split('\t')[0] for line in test_set]

In [48]:
_, _, _, _, _, _, S_eval, T_eval, R = get_source_and_target_matrices(en_to_fr_ents_v2,\
                                                En_shallom_embs_v2, Fr_shallom_embs_v2, given_test_set=test_set, emb_dim=300, test_size=0.1)

Computing S_eval and T_eval: 100%|██████████| 70000/70000 [00:13<00:00, 5037.61it/s]



Now computing R...
Scale S:  6.116433867944561

Completed after 0.4415004253387451 seconds
Alignment loss:  8880.601379922606


In [49]:
hit_values = [1, 5, 10]

In [50]:
_, _, _, _ = greedy_alignment(embed1=(S_eval@R+T_eval)/2, embed2=T_eval, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [68.399 79.807 83.579]%, mr = 53.777, mrr = 0.737583, time = 286.258 s 


In [51]:
_, _, _, _ = greedy_alignment(embed1=T_eval, embed2=(S_eval@R+T_eval)/2, top_k=hit_values, nums_threads=3,\
                                                  metric='euclidean', normalize=False, csls_k=0, accurate=True)

<class 'numpy.ndarray'> float64
accurate results: hits@[1, 5, 10] = [81.809 85.507 86.82 ]%, mr = 469.280, mrr = 0.835872, time = 301.482 s 


In [13]:
evaluate_alignment_knn(S_eval, T_eval, R, assume_known=True, hit_values=[1, 3, 5, 10])

##################################################
Evaluation started...
##################################################
Fitting 1...
Predicting 1...


100%|██████████| 70000/70000 [00:06<00:00, 11037.64it/s]



Fitting 2...
Predicting 2...


100%|██████████| 70000/70000 [00:07<00:00, 9743.21it/s]


Hits@1: 0.7510357142858358, Hits@3: 0.8057857142859514, Hits@5: 0.8265714285717095, Hits@10: 0.8519928571431917, MRR: 0.7867272003535637


In [14]:
scale_S, scale_T, mean_S, mean_T, S, T, R = get_source_and_target_matrices(en_to_fr_ents_v2,\
                                                                                    En_shallom_embs_v2, Fr_shallom_embs_v2, emb_dim=300, test_size=0.0)

Computing S and T: 100%|██████████| 100000/100000 [00:19<00:00, 5137.75it/s]



Now computing R...
Scale S:  6.113047898546291

Completed after 0.695664644241333 seconds
Alignment loss:  28446.94522432553


In [ ]:
evaluate_alignment_knn(S, T, R, assume_known=True, hit_values=[1, 3, 5, 10])

In [15]:
A_neg_S, B_neg_T = get_non_aligned_entity_embedding_matrices(en_to_fr_ents_v2, En_shallom_embs_v2, \
                                                             Fr_shallom_embs_v2, scale_S, scale_T, mean_S, mean_T, emb_dim=300)

Computing A_neg_S...: 0it [00:00, ?it/s]
Computing B_neg_T...: 0it [00:00, ?it/s]


In [16]:
# compute every s_i as (s_i@R+t_i)/2
S = (S@R + T)/2
del T
gc.collect()
Universal_Emb = np.concatenate([A_neg_S@R, B_neg_T, S], axis=0)

In [17]:
np.save('Shallom_EnFr_100K_V2/Universal_Emb.npy', Universal_Emb)

In [18]:
list_merged_entities = sorted(set(En_shallom_embs_v2.index)-set(en_to_fr_ents_v2.keys())) +\
sorted(set(Fr_shallom_embs_v2.index)-set(en_to_fr_ents_v2.values())) + \
list(en_to_fr_ents_v2.keys())
print('Total: ', len(list_merged_entities))
with open('Shallom_EnFr_100K_V2/list_merged_entities_db_fr_en.txt', 'w') as file:
    file.write('\t'.join(list_merged_entities))
del list_merged_entities

Total:  100000


In [20]:
with open('Shallom_EnFr_100K_V2/english2french.txt', 'w') as file:
    json.dump(en_to_fr_ents_v2, file, ensure_ascii=False)